In [14]:
import pickle
import gzip
import json
import ast
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
with open("embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)  # shape (N, D)

with open("item_list.txt") as f:
    item_ids = [line.strip() for line in f]

meta = []
with open("meta_Books.json") as f:
    for line in f:
        try:
            obj = ast.literal_eval(line)
            meta.append(obj)
        except Exception as e:
            print("decode err")
            continue

meta_df = pd.DataFrame(meta)
meta_df = meta_df[["asin", "categories"]]

In [ ]:
df = pd.DataFrame({"asin": item_ids})
df["embedding"] = list(embeddings)

merged = df.merge(meta_df, on="asin", how="inner")


print(len(merged), "produktów z embeddingami i kategoriami")

In [ ]:
def extract_subcategory(cat_list):
    return cat_list[0][1] if len(cat_list) > 0 and len(cat_list[0]) > 1 else "Unknown"

In [ ]:
merged["subcategory"] = merged["categories"].apply(extract_subcategory)
merged = merged.dropna(subset=["subcategory"])